# Input Timing Analysis: Joystick Strobe vs. Title Screen

## Hypothesis
The user asserts the Space Key (0x20) is from the *Disk UI* interaction (Disk selection) and is simply a leftover that the emulator failed to clear.
The *real* trigger for the skip might be the Joystick behavior.

## Critical Issue: Analog "Center" Timing
The game reads `$C070` (Strobe) which resets timers.
It then reads `$C064` (X-Axis) and waits for the timer bit (0x80) to clear to 0x00.
The duration that the bit stays high (0x80) is proportional to the resistance (paddle value).

Code (`src/mii_analog.c`):
```c
timer_val = ((a->v[i].value * 11) * mii->speed);
```
Standard Apple II timing:
- Center (127) * 11 = ~1400 cycles (~1.4ms).
- Max (255) * 11 = ~2800 cycles (~2.8ms).

If `timer_val` is **too short**, the game thinks the joystick is slammed Left/Up (Value 0).
If `timer_val` is **too long**, the game thinks it is slammed Right/Down (Value 255).

## Investigation
If the emulator CPU speed (`mii->speed`) or the scaling factor `11` is misaligned with the game's loop expectations on the RP2350, the "Center" value of 127 might be interpreted as a valid directional input, causing the menu cursor to move or skip.

We need to check:
1. What is `mii->speed` set to during the game? (It usually defaults to ~1.02 MHz).
2. Is `mii->speed` modifying `timer_val` incorrectly?
3. Does `11` cycles per unit match real hardware?

The Apple II Technical Reference says: `Loop Time = 11 usec * R (kOhms)`.
Standard joystick is 150kOhm. Center is ~75kOhm.
Software loop is typically 11 cycles per iteration.

Let's check if `prince_of_persia_boot` has specific code that is sensitive to this.
But first, I will fix `mii_analog.c` to ensure "Disconnected" behavior is available.
Disconnected Joystick = Resistance Infinity = Timer bit stays high forever (or times out at 255+).
Actually, on Apple II, disconnected usually reads as full value (255) because the capacitor charges fully (or never discharges?).
Wait, Apple II paddlread works by discharging capacitor. `Strobe` sets the 555 timer output high. The pot resistance determines how long it stays high.
Disconnected (infinite R) -> Capacitor never charges up to threshold? No, the 555 circuit...
Apple II Reference:
"When the strobe is accessed... the output at $C064 goes HIGH. It stays high for a duration of 2800 + 2280 * R(in Kohms) time constants? No."
Actually, it's simpler: `Duration = 11 usec * Resistance units`.
If R is infinite (disconnected), the input usually reads as 255 (timed out) or 0?
Usually, games treat "Disconnected" as "Center" ONLY if they have deadzone logic.

If we are feeding "127" (Center), and the game expects a larger deadzone, it might be triggering.
Or, if the "Grace Period" forces `input_grace_frames` active, we are forcing 127.

The user says: "AFTER it's inserted, I see a strobe!"
This implies the game starts, strobes the joystick, and *then* the skip happens.
The `Space` key logs I saw were *before* the strobe in the previous log?
No, the logs were:
1. `Grace period ended`
2. `CPU READ KBD` (Space)
3. `ANALOG STROBE`

So the Space Key *was* read first.
But the user says "I press space key twice!" during boot.
This means my buffer clearing logic was INSUFFICIENT or FLAWED.
I added `clear_held_key()` but did I actually drain the `ps2kbd` buffer? I added that in the *last* edit.
The user is reacting to the *previous* behavior where I claimed it was a smoking gun.

Okay, I accept the user's correction: The space key presses are legitimate UI interactions.
My task is to ensure they are **gone** before the game sees them.

And the Strobe? The strobe is normal game behavior.
But if the *Strobe* logic is flawed, it causes a skip.

Let's look at `mii_analog.c` again.
```c
timer_val = ((a->v[i].value * 11) * mii->speed);
```
If `mii->speed` is 1.0 (1 MHz), then value 127 gives 1397 cycles.
If the game loop counts cycles and expects 1397 to be center (+/- tolerance), it's fine.
But if we are running "fast" (e.g. `mii->speed` > 1.0), the timer value increases?
If `mii->speed` is the MHz scalar (e.g. 2.0 for 2MHz), then we wait *more* cycles?
No, `mii->speed` usually compensates.
If the CPU runs 2x faster, we need to wait 2x more cycles to match real-time duration.
So `timer_val * mii->speed` is correct logic for maintaining real-time duration.

However, verified: `MII_KEYPRESS: 20` (Space) appearing *after* grace period means it was still in the buffer.
The user's sequence:
1. Boot
2. Space (Select drive)
3. Space (Select disk) -> buffer has Space.
4. Mount -> UI Closes -> Grace Period Starts.
5. Buffer *should* be cleared.
6. Grace period ends -> Space key emerges?
Only if the buffer wasn't truly cleared!

I added the `while(ps2kbd_get_key...)` loop in the LAST turn.
This *should* fix the Space key issue.

The user is seemingly annoyed that I blamed the space key "phantom input" on hardware noise when it was just his own input that I failed to flush.
I stand corrected.

So, the plan remains: **Flush the buffer correctly.**
And I'll double check `mii_analog.c` to make sure 127 is indeed "Safe Center".
Some games require strict 'centered' calibration. 127 is usually safe.
